In [15]:
import os
import sys
sys.path.append("llmapslab")
from glob import glob
import pandas as pd
from openai_client import client, call_openai_api

import json
import re
import load_configs
import importlib
importlib.reload(load_configs)
from load_configs import (
    openai_api_key,
    llama_api_key, ai21_api_key, 
    gemini_api_key
)
from openai_client import call_openai_api

from langchain_openai import ChatOpenAI
print('pydantic openai_api_key', openai_api_key)  #### openai_api_key.get_secret_value()  will print actual key
os.environ['OPENAI_API_KEY'] = 'api_key_from_env' ####openai_api_key.get_secret_value()

pydantic openai_api_key **********


In [16]:
from langchain_community.llms import OpenAI  ### OpenAI should not be used from community, it is deprecated use langchain_openai
model = OpenAI()

In [5]:
# openai_api_key.get_secret_value()

In [17]:
model.openai_api_key

'api_key_from_env'

In [18]:
model

OpenAI(client=<openai.resources.completions.Completions object at 0x7fb5ee7cc8b0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fb5ed166fd0>, openai_api_key='api_key_from_env', openai_proxy='')

In [12]:
# !pip install langchain_openai

In [9]:
from langchain_openai import OpenAI as LangChainOpenAI

In [10]:
lcmodel = LangChainOpenAI()

In [11]:
lcmodel.openai_api_key

SecretStr('**********')

In [13]:
lcmodel_2 = LangChainOpenAI(openai_api_key="zzxxzzxxzz")

In [14]:
lcmodel_2.openai_api_key

SecretStr('**********')

convert_to_secret_str   - https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/utils.py#L252

openai - https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/llms/base.py

In [ ]:
@root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that api key and python package exists in environment."""
        if values["n"] < 1:
            raise ValueError("n must be at least 1.")
        if values["streaming"] and values["n"] > 1:
            raise ValueError("Cannot stream results when n > 1.")
        if values["streaming"] and values["best_of"] > 1:
            raise ValueError("Cannot stream results when best_of > 1.")

        openai_api_key = get_from_dict_or_env(
            values, "openai_api_key", "OPENAI_API_KEY"
        )
        values["openai_api_key"] = (
            convert_to_secret_str(openai_api_key) if openai_api_key else None
        )
        values["openai_api_base"] = values["openai_api_base"] or os.getenv(
            "OPENAI_API_BASE"
        )
        values["openai_proxy"] = get_from_dict_or_env(
            values, "openai_proxy", "OPENAI_PROXY", default=""
        )
        values["openai_organization"] = (
            values["openai_organization"]
            or os.getenv("OPENAI_ORG_ID")
            or os.getenv("OPENAI_ORGANIZATION")
        )

        client_params = {
            "api_key": (
                values["openai_api_key"].get_secret_value()
                if values["openai_api_key"]
                else None
            ),
            "organization": values["openai_organization"],
            "base_url": values["openai_api_base"],
            "timeout": values["request_timeout"],
            "max_retries": values["max_retries"],
            "default_headers": values["default_headers"],
            "default_query": values["default_query"],
        }
        if not values.get("client"):
            sync_specific = {"http_client": values["http_client"]}
            values["client"] = openai.OpenAI(
                **client_params, **sync_specific
            ).completions
        if not values.get("async_client"):
            async_specific = {"http_client": values["http_async_client"]}
            values["async_client"] = openai.AsyncOpenAI(
                **client_params, **async_specific
            ).completions
